# Colab only
❗ This notebook is designed to run on google colab.   
🎉 These top few cells should install the nescesary libraries needed and set you up to be able to run the rest of the notebook!

You should already have a google account ready to go!

To bore you with details, these cells will:
- Install needed packages which are not already installed google-colab
- Mount your google drive ready to be used 

### 🧙‍♀️ Wizards
🧙‍♂️ If you're a wizard, here's some info about colab you may want to know:
- Each colab notebook runs on its own temporary linux virtual machine with its own filesystem.   
- If your notebook is shutdown, this will delete the temporary instance - this is why you need to mount google drive
- Colab seems to let you have about 3 notebook instances running at any time; each of these will be on their own unique VM

In [ ]:
# Mounting google drive
# Follow the prompts with default settings
import os
from os.path import join

from google.colab import drive
google_dir = '/content/drive'
drive.mount(google_dir)

In [ ]:
from pathlib import Path
project_dir = Path(google_dir) / 'MyDrive' / "ASDAF_workshop_data"
storage_location = project_dir / "workshop_data"

storage_location.mkdir(exist_ok=True, parents=True)

# Now list the contents to see whats in it. If this is the first time you've run this,
# "storage_location" may be empty!
!ls {project_dir}
!ls {storage_location}

In [ ]:
# Now ensure the data we need is in the correct place
!wget --output-document shp_data.zip https://github.com/AustralianSDAF/ASDAF_Workshop_Materials_Remote_Sensing_Intro/releases/download/shapefile_data_v1.0/john_forrest_rough.zip  
!unzip shp_data.zip -d ../shp_data
# Now list the contents to make sure we see the 4 shapefile components (.dbf, .prj, .shp, and .shx)
!ls -al ../shp_data


In [ ]:
# This cell has been automatically inserted from build/google_colab_cell.py
# It should make this notebook google-colab compatible!

!pip install --upgrade pip 
!pip install folium
!pip install rioxarray
!pip install geopandas
!pip install earthpy
!pip install numba
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install xarray
!pip install tqdm
!pip install pystac-client

# Custom Raster calculations using Numba
## ❓ Questions
- How do I create fast custom raster calculations that have more flexibility than basic operations?


## ❗ Objectives
- Create a `numba` compiled function that operates on multiple single pixels.
- Create a `numba` compiled function that operates on multiple pixels at once.


# What is Numba?
Python has several features that make it a great prototyping language, but it's native speed is often criticised. While this is ultimately down to the implementation, it is still an issue.  

Numpy and several other libraries will wrap compiled C, C++, or Fortran code to try to have the benefits of Python without giving up too much speed.  

[Numba](https://numba.pydata.org/) takes a different approach. When you wrap a Python function with a `numba` compilation 'decorator', it will try to compile it using the state-of-the-art LLVM compiler. 
One drawback is that it will only run with a subset of Python and `numpy` functionality - most libraries will not be able to be compiled.
One of the better features of `numba` is that you can write loops and the compiler will automatically optimise your code to be as fast or faster than single line `numpy` functions.  

"
You don't need to replace the Python interpreter, run a separate compilation step, or even have a C/C++ compiler installed. Just apply one of the Numba decorators to your Python function, and Numba does the rest. 
"

# Example function
Let's write an example function in `numba` to get the hang of it.

In [ ]:
from numba import njit
import numpy as np




In [ ]:
# Timing the code


Note that while `numba` can create fast custom calculations, your implementation may not always be as fast as an existing implementation due to a number of factors.

# Applying numba to remote sensing data
Let's load back in our data and create a function to apply to it.

In [ ]:
# Get the directory again
import os
from os.path import join

product_dir_textfile = "product_dir.txt"

with open(product_dir_textfile, 'r') as f:
    base_product_dir = f.readline()

product_dir = join(base_product_dir, 'GRANULE')
L2_dirname = os.listdir(product_dir)[0]
product_dir = join(product_dir, L2_dirname)
product_dir = join(product_dir, 'IMG_DATA', 'R60m')

# Get a dictionary of filenames
image_paths = {}
for fname in os.listdir(product_dir):
    fpath = join(product_dir, fname)
    file_band = fname.split('_')[2]
    image_paths[file_band] = fpath

image_paths

In [ ]:
import rioxarray

red_int = rioxarray.open_rasterio(image_paths['B04'])
nir_int = rioxarray.open_rasterio(image_paths['B8A'])

In [ ]:
# What data type is it?
red_int.dtype

Let's convert the `uint16` data to `float32`. To do this we will need to take the integer and divide by 10000 to get a reflectance.

In [ ]:
red = red_int.astype(np.float32)/10000
nir = ndvi_int.astype(np.float32)/10000
red.dtype

## NDVI
For some more practice, let's reimplement the NDVI using `numba`.


Time this code and compare it to the previous way of creating an NDVI. Is it worth it?

# Apply a kernel
We just applied a single pixel operation. Let's now use a more real-world use case, applying a kernel over an image.